In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator and then re-execute this cell.')
else:
    print(gpu_info)

Wed Jun 23 13:32:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory

ram = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM.'.format(ram))
if ram < 20:
    print('To enable a high-RAM runtime, select the Runtime > "Change runtime type", then select High-RAM in the Runtime shape dropdown '
          'and then re-execute this cell.')
else:
    print('You are using a high-RAM runtime.')

Your runtime has 13.6 gigabytes of available RAM.
To enable a high-RAM runtime, select the Runtime > "Change runtime type", then select High-RAM in the Runtime shape dropdown and then re-execute this cell.


In [4]:
from tensorflow import config

physical_devices = config.list_physical_devices('GPU')
try:
    config.experimental.set_memory_growth(physical_devices[0], True)
except Exception as exception:
    print(exception)

In [5]:
!pip install tensorflow-addons

     |████████████████████████████████| 686kB 12.5MB/s 


In [6]:
from tensorflow import device
from tensorflow_addons import layers as new_layers
from tensorflow.keras import losses
from tensorflow.keras import models

import gc
import h5py
import imageio
import numpy as np
import os

In [7]:
model_path = '/content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/checkpoints/UNet12/model_39.h5'
with device('gpu:0'):
    model = models.load_model(model_path)
loss = losses.MeanSquaredError()

dynamic_file_path = '/content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/files/testing/2019-07-02_test.h5'
static_file_path = '/content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/files/ISTANBUL_static_2019.h5'

In [8]:
def get_data(file_path):
    file = h5py.File(file_path, 'r')
    group_key = list(file.keys())[0]
    data = np.array(file[group_key][:], dtype=np.float32)
    file.close()
    return data

In [9]:
def get_mask(data):
    data = data.reshape(-1, 495, 436, 9)
    data = np.take(data, [1, 3, 5, 7], axis=-1)
    data = np.moveaxis(data, -1, 1).reshape(-1, 495, 436)
    data = np.mean(data, axis=0, dtype=np.float32)
    return (data > 0.0).astype(np.uint8)

In [10]:
def create_movie(winner_prediction, prediction, path):
    winner_prediction = winner_prediction.reshape((-1, 495, 436, 8))
    winner_prediction = np.sum(winner_prediction, axis=-1, dtype=np.uint8)
    prediction = prediction.reshape((-1, 495, 436, 8))
    prediction = np.sum(prediction, axis=-1, dtype=np.uint8)
    prediction *= mask
    frames = np.concatenate((winner_prediction, prediction), axis=-1)
    imageio.mimsave(path, [frame for frame in frames], fps=3)
    print('created movie:', path)

In [11]:
winner_prediction = get_data('/content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/2019-07-02_test.h5')
dynamic_data = get_data(dynamic_file_path)
static_data = get_data(static_file_path)

In [12]:
dynamic_data = np.moveaxis(dynamic_data, 1, -1).reshape((3, 495, 436, -1))
mask = get_mask(dynamic_data)
static_data = np.repeat(np.expand_dims(static_data, axis=0), repeats=3, axis=0)
data = np.concatenate((dynamic_data, static_data), axis=-1)
with device('gpu:0'):
    prediction = model.predict(data)
    prediction = np.clip(prediction, 0.0, 1.0)
gc.collect()
prediction = np.moveaxis(prediction.reshape((3, 495, 436, 8, 12)), -1, 1)
prediction = np.take(prediction, [0, 1, 2, 5, 8, 11], axis=1)
prediction *= 255.0

In [13]:
loss(winner_prediction, prediction).numpy()

9.968577

In [14]:
create_movie(winner_prediction, prediction, '/content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/2019-07-02_test.gif')

created movie: /content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/2019-07-02_test.gif
